In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from platform import python_version
print(python_version())

3.6.13


In [3]:
import sys
print(sys.executable)

/home/mayank/.conda/envs/w2v2/bin/python


In [3]:
!unset CUDA_VISIBLE_DEVICES
!export CUDA_VISIBLE_DEVICES=1

In [14]:
!nvidia-smi
!echo $CUDA_VISIBLE_DEVICES

Tue Feb  1 07:25:58 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:3B:00.0 Off |                  N/A |
| 58%   81C    P2    95W / 250W |   7181MiB / 11178MiB |     98%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:5E:00.0 Off |                  N/A |
| 59%   

In [15]:
# %%capture

# !pip install git+https://github.com/huggingface/datasets.git
# !pip install git+https://github.com/huggingface/transformers.git
# !pip install jiwer
# !pip install torchaudio
# !pip install librosa

# # Monitor the training process
# # !pip install wandb

In [3]:
%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8
%env TRANSFORMERS_CACHE=content/cache
%env HF_DATASETS_CACHE=content/cache
%env CUDA_LAUNCH_BLOCKING=1
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
env: TRANSFORMERS_CACHE=content/cache
env: HF_DATASETS_CACHE=content/cache
env: CUDA_LAUNCH_BLOCKING=1
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [4]:
import os
print(os.environ['CUDA_VISIBLE_DEVICES'])
os.environ['CUDA_VISIBLE_DEVICES']='1'
print(os.environ['CUDA_VISIBLE_DEVICES'])

0
1


In [2]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split

import os, sys, json
import librosa

/home/aman/anaconda3/envs/w2v2/lib/python3.6/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [9]:
# data = []
# json_folder = '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/jsons/'
# jsons = [f.name for f in os.scandir(json_folder) if 'json' in f.name and f.name.split('.')[0] not in ['unlabelled', 'other']]

# print(jsons)
# for accent in jsons:
#     print("loading", accent)
#     json_path = json_folder + accent
#     json_file = open(json_path)
#     json_item_list = [line for line in json_file]
#     json_item_list = json_item_list[:4000]
#     json_item_list = [json.loads(line.strip()) for line in json_item_list]
#     for sample in tqdm(json_item_list):
#         try:
#             path = sample['audio_filepath']
#             name = str(path).split('/')[-1].split('.')[0]
#             label = sample['accent']
#             duration = librosa.get_duration(filename=path)
#             if duration > 30:
#                 continue
#             data.append({
#                 "name": name,
#                 "path": path,
#                 "accent": label
#             })
#         except Exception as e:
#             print(str(path), e)
#             pass

In [32]:
from multiprocessing import Pool, Manager

def process_jsons(accent):
    print("loading", accent)
    json_path = json_folder + accent
    json_file = open(json_path)
    json_item_list = [line for line in json_file]
    json_item_list = json_item_list[:1000]
    json_item_list = [json.loads(line.strip()) for line in json_item_list]
    for sample in tqdm(json_item_list):
        try:
            path = sample['audio_filepath']
            name = str(path).split('/')[-1].split('.')[0]
            label = sample['accent']
            duration = librosa.get_duration(filename=path)
            if duration > 30:
                continue
            L.append({
                "name": name,
                "path": path,
                "accent": label
            })
        except Exception as e:
            print(str(path), e)
            pass
#         break

json_folder = '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/jsons/'
jsons = [f.name for f in os.scandir(json_folder) if 'json' in f.name and f.name.split('.')[0] not in ['unlabelled', 'other']]

# p = Pool(16)
# with Manager() as manager:
#     L = manager.list()
#     with p:
#         p.map(process_jsons, jsons)
#     print(L)


manager = Manager()
L = manager.list()
pool = Pool(16)
pool.map(process_jsons, jsons)
# tqdm(pool.imap(process_jsons, jsons), len(jsons))
pool.close()
pool.join()
print(len(L))

loadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloadingloading          bermuda.jsonaustralia.json   philippines.json  hongkong.jsonwales.json malaysia.jsonireland.jsonsouthatlandtic.jsonnewzealand.jsonus.jsonsingapore.json
indian.jsonscotland.json

canada.jsonafrican.json


england.json











100%|██████████| 1000/1000 [10:04<00:00,  1.65it/s]


13733


In [33]:
data = list(L)
print(len(data))
print(data[:3])

13733
[{'name': 'common_voice_en_20538780', 'path': '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/mozilla/cv-corpus-7.0-2021-07-21/en/clips/common_voice_en_20538780.mp3', 'accent': 'ireland'}, {'name': 'common_voice_en_26963927', 'path': '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/mozilla/cv-corpus-7.0-2021-07-21/en/clips/common_voice_en_26963927.mp3', 'accent': 'bermuda'}, {'name': 'common_voice_en_27036316', 'path': '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/mozilla/cv-corpus-7.0-2021-07-21/en/clips/common_voice_en_27036316.mp3', 'accent': 'malaysia'}]


In [34]:
df = pd.DataFrame(data)
# df = df.sample(frac=1).reset_index(drop=True)
df.head()

,name,path,accent
0,common_voice_en_20538780,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,ireland
1,common_voice_en_26963927,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,bermuda
2,common_voice_en_27036316,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,malaysia
3,common_voice_en_18803681,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,hongkong
4,common_voice_en_24681430,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,african


In [35]:
## Filter broken and non-existed paths

print(f"Step 0: {len(df)}")

df["status"] = df["path"].apply(lambda path: True if os.path.exists(path) else None)
df = df.dropna(subset=["path"])
df = df.drop("status", 1)
print(f"Step 1: {len(df)}")

# df = df.sample(frac=1)
# df = df.reset_index(drop=True)
df.head()

Step 0: 13733
Step 1: 13733


,name,path,accent
0,common_voice_en_20538780,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,ireland
1,common_voice_en_26963927,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,bermuda
2,common_voice_en_27036316,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,malaysia
3,common_voice_en_18803681,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,hongkong
4,common_voice_en_24681430,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,african


In [36]:
print("Labels: ", df["accent"].unique())
print()
df.groupby("accent").count()[["path"]]

Labels:  ['ireland' 'bermuda' 'malaysia' 'hongkong' 'african' 'singapore'
 'newzealand' 'australia' 'philippines' 'indian' 'wales' 'southatlandtic'
 'canada' 'scotland' 'england' 'us']



,path
accent,
african,1000
australia,1000
bermuda,248
canada,1000
england,1000
hongkong,1000
indian,1000
ireland,1000
malaysia,1000


Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.

In [37]:
import torchaudio
import librosa
import IPython.display as ipd
import numpy as np

idx = np.random.randint(0, len(df))
sample = df.iloc[idx]
path = sample["path"]
label = sample["accent"]


print(f"ID Location: {idx}")
print(f"      Label: {label}")
print()

speech, sr = torchaudio.load(path)

print(sr)
# speech = speech[0].numpy().squeeze()
# speech = librosa.resample(np.asarray(speech), sr, 16_000)
# ipd.Audio(data=np.asarray(speech), autoplay=True, rate=16000)

ID Location: 2661
      Label: malaysia

48000


In [38]:
!ls

accent-classification.ipynb	     dristi_accent_classifier_using_wav2vec2.py
accent_classifier_using_wav2vec2.py  jsons
data


In [39]:
!mkdir -p data/with_all_1k

For training purposes, we need to split data into train test sets; in this specific example, we break with a `20%` rate for the test set.

In [40]:
save_path = "data/with_all_1k/"

train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["accent"])

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)


print(train_df.shape)
print(test_df.shape)

(10986, 3)
(2747, 3)


## Prepare Data for Training

In [41]:
# Loading the created dataset using datasets
from datasets import load_dataset, load_metric

data_files = {
    "train": "data/with_all_1k/train.csv", 
    "validation": "data/with_all_1k/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Using custom data configuration default-5b6ada385356f15d


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to content/cache/csv/default-5b6ada385356f15d/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['name', 'path', 'accent'],
    num_rows: 10986
})
Dataset({
    features: ['name', 'path', 'accent'],
    num_rows: 2747
})


In [42]:
# We need to specify the input and output column
input_column = "path"
output_column = "accent"

In [43]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 16 classes: ['african', 'australia', 'bermuda', 'canada', 'england', 'hongkong', 'indian', 'ireland', 'malaysia', 'newzealand', 'philippines', 'scotland', 'singapore', 'southatlandtic', 'us', 'wales']


In [44]:
from transformers import AutoConfig, Wav2Vec2Processor

In [55]:
# model_name_or_path = "facebook/wav2vec2-large-lv60" #"facebook/wav2vec2-base-960h" #"facebook/wav2vec2-large-960h" #"facebook/wav2vec2-base"
model_name_or_path = "facebook/wav2vec2-base" #"facebook/wav2vec2-base-960h" #"facebook/wav2vec2-large-960h" #"facebook/wav2vec2-base"
pooling_mode = "mean"

In [56]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="wav2vec2_clf",
)
setattr(config, 'pooling_mode', pooling_mode)

Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/transformers/configuration_utils.py:349: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [57]:
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"The target sampling rate: {target_sampling_rate}")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

The target sampling rate: 16000


# Preprocess Data

In [58]:
def speech_file_to_array_fn(path):
    speech_array, sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech

def label_to_id(label, label_list):
    if len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1

    return label

def preprocess_function(examples):
    speech_list = [speech_file_to_array_fn(path) for path in examples[input_column]]
    target_list = [label_to_id(label, label_list) for label in examples[output_column]]

    result = processor(speech_list, sampling_rate=target_sampling_rate)
    result["labels"] = list(target_list)

    return result

In [59]:
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=16
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=16
)

/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-t

/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-t

In [60]:
idx = 0
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['accent']}")

Training input_values: [-0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232836917, -0.0001295561232

## Model



In [17]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [20]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)


class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception("The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [21]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch

import transformers
from transformers import Wav2Vec2Processor

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch

In [22]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [23]:
is_regression = False

In [24]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [25]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)
# this will give a warning which is totally fine

The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [68]:
model.freeze_feature_extractor()

In [69]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="csvs/w2v2-accent-recognition",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1.0,
    fp16=True,
    save_steps=800,
    eval_steps=800,
    logging_steps=800,
    learning_rate=1e-4,
    save_total_limit=2,
)

For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [70]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)
        
#         import pdb
#         pdb.set_trace()

        if self.use_amp:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_amp:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()

Now, all instances can be passed to Trainer and we are ready to start training!

In [71]:
trainer = CTCTrainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook. 

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [72]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `Wav2Vec2ForSpeechClassification.forward` and have been ignored: name, accent, path.
***** Running training *****
  Num examples = 10986
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 2
  Total optimization steps = 1373


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

## Evaluation

In [ ]:
# need to save the model first
import librosa
from sklearn.metrics import classification_report

In [ ]:
test_dataset = load_dataset("csv", data_files={"test": "csvs/data/test.csv"}, delimiter="\t")["test"]
test_dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [7]:
model_name_or_path = "/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/csvs/accent-recognition/checkpoint-4000/"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

OSError: Can't load tokenizer for '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/csvs/accent-recognition/checkpoint-4000/'. Make sure that:

- '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/csvs/accent-recognition/checkpoint-4000/' is a correct model identifier listed on 'https://huggingface.co/models'
  (make sure '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/csvs/accent-recognition/checkpoint-4000/' is not a path to a local directory with something else, in that case)

- or '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/csvs/accent-recognition/checkpoint-4000/' is the correct path to a directory containing relevant tokenizer files



In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, processor.feature_extractor.sampling_rate)

    batch["speech"] = speech_array
    return batch


def predict(batch):
    features = processor(batch["speech"], sampling_rate=processor.feature_extractor.sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits 

    pred_ids = torch.argmax(logits, dim=-1).detach().cpu().numpy()
    batch["predicted"] = pred_ids
    return batch

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn)

In [ ]:
result = test_dataset.map(predict, batched=True, batch_size=8)

In [ ]:
label_names = [config.id2label[i] for i in range(config.num_labels)]
label_names

In [ ]:
y_true = [config.label2id[name] for name in result["accent"]]
y_pred = result["predicted"]

print(y_true[:5])
print(y_pred[:5])

In [ ]:
print(classification_report(y_true, y_pred, target_names=label_names))

# Prediction

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
from transformers import AutoConfig, Wav2Vec2Processor

import librosa
import IPython.display as ipd
import numpy as np
import pandas as pd

In [ ]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput

@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

In [4]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, **kwargs):
        x = features
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        print('----------------------------')
        print(x)
        print(x.shape)
        print('----------------------------')
        x = self.out_proj(x)
        return x


class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception("The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")

        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model_name_or_path = "/mnt/data/aman/mayank/MTP/mount_points/jan_19/Error-Driven-ASR-Personalization/MCV_accent/data/dristi_accent-recognition/checkpoint-6400/"
model_name_or_path = "/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/MCV_accent/data/dristi_accent-recognition/checkpoint-6400/"
config = AutoConfig.from_pretrained(model_name_or_path)
processor = Wav2Vec2Processor.from_pretrained(model_name_or_path)
sampling_rate = processor.feature_extractor.sampling_rate
model = Wav2Vec2ForSpeechClassification.from_pretrained(model_name_or_path).to(device)

In [7]:
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

Model's state_dict:
wav2vec2.masked_spec_embed 	 torch.Size([768])
wav2vec2.feature_extractor.conv_layers.0.conv.weight 	 torch.Size([512, 1, 10])
wav2vec2.feature_extractor.conv_layers.0.layer_norm.weight 	 torch.Size([512])
wav2vec2.feature_extractor.conv_layers.0.layer_norm.bias 	 torch.Size([512])
wav2vec2.feature_extractor.conv_layers.1.conv.weight 	 torch.Size([512, 512, 3])
wav2vec2.feature_extractor.conv_layers.2.conv.weight 	 torch.Size([512, 512, 3])
wav2vec2.feature_extractor.conv_layers.3.conv.weight 	 torch.Size([512, 512, 3])
wav2vec2.feature_extractor.conv_layers.4.conv.weight 	 torch.Size([512, 512, 3])
wav2vec2.feature_extractor.conv_layers.5.conv.weight 	 torch.Size([512, 512, 2])
wav2vec2.feature_extractor.conv_layers.6.conv.weight 	 torch.Size([512, 512, 2])
wav2vec2.feature_projection.layer_norm.weight 	 torch.Size([512])
wav2vec2.feature_projection.layer_norm.bias 	 torch.Size([512])
wav2vec2.feature_projection.projection.weight 	 torch.Size([768, 512])
wav2vec2.f

wav2vec2.encoder.layers.7.attention.out_proj.weight 	 torch.Size([768, 768])
wav2vec2.encoder.layers.7.attention.out_proj.bias 	 torch.Size([768])
wav2vec2.encoder.layers.7.layer_norm.weight 	 torch.Size([768])
wav2vec2.encoder.layers.7.layer_norm.bias 	 torch.Size([768])
wav2vec2.encoder.layers.7.feed_forward.intermediate_dense.weight 	 torch.Size([3072, 768])
wav2vec2.encoder.layers.7.feed_forward.intermediate_dense.bias 	 torch.Size([3072])
wav2vec2.encoder.layers.7.feed_forward.output_dense.weight 	 torch.Size([768, 3072])
wav2vec2.encoder.layers.7.feed_forward.output_dense.bias 	 torch.Size([768])
wav2vec2.encoder.layers.7.final_layer_norm.weight 	 torch.Size([768])
wav2vec2.encoder.layers.7.final_layer_norm.bias 	 torch.Size([768])
wav2vec2.encoder.layers.8.attention.k_proj.weight 	 torch.Size([768, 768])
wav2vec2.encoder.layers.8.attention.k_proj.bias 	 torch.Size([768])
wav2vec2.encoder.layers.8.attention.v_proj.weight 	 torch.Size([768, 768])
wav2vec2.encoder.layers.8.attentio

In [8]:
model.classifier.dense

Linear(in_features=768, out_features=768, bias=True)

In [13]:
def speech_file_to_array_fn(path, sampling_rate):
    speech_array, _sampling_rate = torchaudio.load(path)
    resampler = torchaudio.transforms.Resample(_sampling_rate)
    speech = resampler(speech_array).squeeze().numpy()
    return speech


def predict(path, sampling_rate):
    speech = speech_file_to_array_fn(path, sampling_rate)
    features = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = features.input_values.to(device)
#     attention_mask = features.attention_mask.to(device)
#### just trying ####
    attention_mask = None
#### just trying ####

    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits

    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Accent": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


STYLES = """
<style>
div.display_data {
    margin: 0 auto;
    max-width: 500px;
}
table.xxx {
    margin: 50px !important;
    float: right !important;
    clear: both !important;
}
table.xxx td {
    min-width: 300px !important;
    text-align: center !important;
}
</style>
""".strip()

def prediction(df_row):
    path, accent = df_row["path"], df_row["accent"]
    df = pd.DataFrame([{"Accent": accent, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    ipd.display(ipd.HTML(STYLES + df.to_html(**setup) + "<br />"))
    speech, sr = torchaudio.load(path)
    speech = speech[0].numpy().squeeze()
    speech = librosa.resample(np.asarray(speech), sr, sampling_rate)
    ipd.display(ipd.Audio(data=np.asarray(speech), autoplay=True, rate=sampling_rate))

    outputs = predict(path, sampling_rate)
    r = pd.DataFrame(outputs)
    ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [14]:
test = pd.read_csv("data/with_all_1k/test.csv", sep="\t")
test.head()

,name,path,accent
0,common_voice_en_37020,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,wales
1,common_voice_en_19046599,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,ireland
2,common_voice_en_17581289,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,bermuda
3,common_voice_en_18558116,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,us
4,common_voice_en_24997856,/home/mayank/MTP/begin_again/Error-Driven-ASR-...,philippines


In [15]:
test.iloc[3]

name                               common_voice_en_18558116
path      /home/mayank/MTP/begin_again/Error-Driven-ASR-...
accent                                                   us
Name: 3, dtype: object

In [18]:
prediction(test.iloc[3])

,Accent,Sentence
0,us,


----------------------------
tensor([[ 0.6380,  0.4538, -0.6216, -0.1487, -0.2259,  0.4072, -0.3208, -0.3406,
         -0.0744,  0.4975,  0.6141, -0.6491, -0.7860,  0.7984, -0.0450,  0.7170,
          0.3164, -0.8108,  0.2395,  0.9135, -0.8438, -0.6810,  0.3190, -0.7073,
          0.4431,  0.6747, -0.8412, -0.8838,  0.3657, -0.6211, -0.5514,  0.5275,
          0.1765, -0.2143,  0.0710, -0.3310,  0.3699, -0.5275,  0.4960, -0.0243,
         -0.5171, -0.0187,  0.4351,  0.2703, -0.6955,  0.3754,  0.8166, -0.5852,
         -0.3795, -0.5020, -0.0746, -0.7623,  0.5649,  0.1282, -0.5238,  0.5795,
         -0.0567,  0.7104,  0.4449, -0.0055, -0.6379,  0.8639, -0.1471, -0.6576,
          0.5014, -0.7628, -0.8134, -0.4666,  0.8642, -0.1523,  0.7685, -0.8991,
          0.7981,  0.2253, -0.0217, -0.5101,  0.7464,  0.7537, -0.6667, -0.7466,
         -0.6503, -0.0338, -0.6900, -0.2404,  0.6847, -0.6509,  0.2080, -0.3107,
         -0.2071,  0.3492,  0.7795,  0.3154,  0.6850,  0.7272,  0.6920, -0.1145,

,Accent,Score
0,african,0.0%
1,australia,0.0%
2,bermuda,0.0%
3,canada,0.0%
4,england,0.0%
5,hongkong,0.0%
6,indian,0.0%
7,ireland,0.0%
8,malaysia,0.0%
9,newzealand,0.0%


In [19]:
print(test.iloc[1])
prediction(test.iloc[1])

name                               common_voice_en_19046599
path      /home/mayank/MTP/begin_again/Error-Driven-ASR-...
accent                                              ireland
Name: 1, dtype: object


,Accent,Sentence
0,ireland,


----------------------------
tensor([[-0.3604, -0.5015,  0.5872, -0.7396, -0.8553, -0.8409, -0.3025, -0.6030,
          0.5019, -0.7891, -0.2193,  0.7910, -0.6881, -0.4744,  0.7632,  0.2645,
         -0.5625, -0.4870, -0.7536,  0.7234,  0.7052,  0.4127,  0.8077,  0.8328,
         -0.5481, -0.8586, -0.2990,  0.5092,  0.2156, -0.5565, -0.5920, -0.1352,
          0.4027, -0.6018,  0.6802, -0.3259, -0.8174, -0.3766, -0.0789,  0.4465,
          0.7930,  0.9042, -0.7596, -0.7964, -0.7144,  0.5105, -0.4559, -0.0338,
         -0.6765, -0.1597, -0.8678, -0.4116,  0.3043,  0.5955, -0.1498,  0.4865,
         -0.6807, -0.6949,  0.4006,  0.5696, -0.7316, -0.8384,  0.2837, -0.4042,
         -0.8144,  0.7488,  0.7497,  0.5894, -0.1367, -0.8820, -0.2325, -0.5952,
         -0.8967, -0.8156, -0.6474,  0.8483, -0.5507,  0.6613,  0.8286,  0.4074,
          0.8415,  0.7009, -0.3253, -0.3987, -0.7224, -0.6438, -0.1427,  0.0512,
         -0.8593, -0.4745, -0.3878,  0.0240, -0.0655, -0.3107,  0.5302, -0.3431,

,Accent,Score
0,african,0.0%
1,australia,0.0%
2,bermuda,0.0%
3,canada,0.0%
4,england,0.0%
5,hongkong,0.0%
6,indian,0.0%
7,ireland,100.0%
8,malaysia,0.0%
9,newzealand,0.0%


In [ ]:
prediction(test.iloc[2])

In [24]:
test.iloc[1]['path']
f1 = '/home/mayank/MTP/begin_again/Error-Driven-ASR-Personalization/mozilla/cv-corpus-7.0-2021-07-21/en/clips/common_voice_en_19046599.mp3'

In [25]:
import numpy as np
import json
from tqdm import tqdm
import pickle
import librosa
import torch
import torchaudio
from transformers import Wav2Vec2Tokenizer, Wav2Vec2Model

device = torch.device('cuda')

to = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h")
mo = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-large-960h").to(device)
layer_num = 1

# waveform, sample_rate = torchaudio.load(test.iloc[3]['path'])
waveform, sample_rate = librosa.load(f1, sr=48000)
# print(sample_rate)
waveform = torch.tensor(waveform)
speech_inp = to(waveform, return_tensors='pt', padding='longest').input_values.to(device)
w2v2_features = mo(speech_inp, output_hidden_states=True).hidden_states[layer_num - 1].mean(1).cpu().detach().numpy()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
/home/mayank/.conda/envs/w2v2/lib/python3.6/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:423: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of the model checkpoint at facebook/wav2vec2-large-960h were not used when initializing Wav2Vec2Model: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS 

In [26]:
w2v2_features.shape

(1, 1024)

In [28]:
print(w2v2_features)

[[ 0.03266818 -0.26813987 -0.04880898 ... -0.49008036  0.20770192
   0.20002435]]
